# Handling product knowledge Q&A with pre-processing

We were given [a spreadsheet](https://www.dropbox.com/scl/fi/4ls998569fgbo9zjn5tpv/matrix_expertos.xlsx?rlkey=ktd3hchpei60q4pm3o1c31lal&dl=0) containing a matrix of concrete products and appropriate applications for them.

The challenge is to answer natural-language questions about the products in the spreadsheet, which is very large and complex.

We take a multi-stage approach using a lengthy pre-processing step.

First install all our deps:
* MongoDB as our vector store
* LlamaParse for parsing the spreadsheet
* Anthropic for understanding the spreadsheet
* OpenAI embeddings to embed the data

In [ ]:
!pip install llama-index-core llama-cloud llama-cloud-services llama-index-llms-anthropic llama-index-indices-managed-llama-cloud

Connect to our llamacloud index:

In [ ]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
from google.colab import userdata

index = LlamaCloudIndex(
  name="mongodb-cemex-demo",
  project_name="Rando project",
  organization_id="e793a802-cb91-4e6a-bd49-61d0ba2ac5f9",
  api_key=userdata.get('llamacloud-cemex')
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

We parse the spreadsheet into a single markdown document, which LLMs find easier to understand:

In [ ]:
from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader

parser = LlamaParse(
    result_type="markdown",
    api_key = userdata.get('llamacloud-cemex')
)
documents = parser.load_data('data/matrix_expertos.xlsx')

Started parsing the file under job_id e6cc8852-f581-4662-8528-e32e70fe164a


We get a very large Markdown table:

In [ ]:
raw_output = documents[0].text
print(raw_output)

---
# elementos

| | | | |Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Productos|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Soluciones|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|Mutiproductos|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Subsegmento|Especialidad|Tipo de obra|Elementos|Aparentia Arquitectónico|Duramax|Duramax Autosellante|Biocrete Antibac|Biocrete Antit

We confirm that Claude 3.7 Sonnet is capable of understanding what it's looking at by asking it to translate the headers:

In [ ]:
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(
    model="claude-3-7-sonnet-20250219",
    api_key=userdata.get('anthropic-key')
)

In [ ]:
response = llm.complete(f"You are looking at a spreadsheet in the form of a large markdown table. Translate the labels into english. <spreadsheet>{raw_output}</spreadsheet>")

In [ ]:
print(response)

# Translation of Spreadsheet Labels

I've translated the column headers and main section labels from Spanish to English:

## Column Headers
- "Subsegmento" -> "Subsegment"
- "Especialidad" -> "Specialty"
- "Tipo de obra" -> "Type of work"
- "Elementos" -> "Elements"
- "Productos" -> "Products"
- "Soluciones" -> "Solutions"
- "Mutiproductos" -> "Multi-products"

## Main Section Labels
- "elementos" -> "elements"
- "Comercio" -> "Commerce"
- "Edificación Vertical" -> "Vertical Building"
- "Estacionamiento" -> "Parking"
- "Losas" -> "Slabs"
- "Trabes" -> "Beams"
- "Columnas" -> "Columns"
- "Muros" -> "Walls"
- "Obras hidráulicas/Sanitarias" -> "Hydraulic/Sanitary Works"
- "Plantas de tratamiento" -> "Treatment Plants"
- "Cisternas" -> "Cisterns"
- "Bases" -> "Bases"
- "Canales" -> "Channels"
- "Pozos de visita" -> "Manholes"
- "Registros" -> "Registers"
- "Relleno de zanjas / Encofrados" -> "Trench Filling / Formwork"
- "Obras exteriores" -> "Exterior Works"
- "Motor Lobby" -> "Motor Lobb

We now use Claude to transform the data from spreadsheet rows and columns to a series of declarative statements, one for each property in the spreadsheet. This greatly expands the volume of data being processed but makes it much more amenable to semantic search, since the meaning of each "X" is expanded into its plain semantic meaning (in Spanish).

This involves several subtleties:
* We need to extend our `max_tokens` to allow the longest possible output from the model
* The output is still much longer than `max_tokens`, so we modify the prompt to focus on one section of the spreadsheet at a time (there are conveniently four top-level subsegments we can use for this purpose)
* Though the instructions are in English the requested output remains in Spanish since the expected questions are in that language
* Processing this step takes about 20 minutes

In [ ]:
long_response_llm = llm = Anthropic(
    model="claude-3-7-sonnet-20250219",
    api_key=userdata.get('anthropic-high-volume'),
    max_tokens=64000
)

In [ ]:
subsegments = ["Commercial","Industrial","Infrastructure","Housing"]

all_responses = ""
for subsegment in subsegments:
  response = long_response_llm.complete(f"""
    You are given a large table derived from a spreadsheet.

    The first four columns define a possible place a product could be used, aka their applications. They are:
    - **Subsegmento** -> Subsegment (commercial, industrial, infrastructure, housing)
    - **Especialidad** -> Specialty (hotels, supermarkets, healthcare, industrial parks, etc.)
    - **Tipo de obra** -> Type of work (parking, hydraulic/sanitary work, exteriors, etc.)
    - **Elementos** -> Elements (such as slabs, beams, columns, walls, etc.)

    The remaining columns are products, solutions, and "multi-products".

    If the product, solution or multi-product is appropriate for the application, there is an X or an x in the corresponding column.

    Convert the table into a series of statements (in Spanish), e.g.
    "Biocrete Antihongo-Antialga es apropiado para bases hidráulicas/sanitarias en edificaciones verticales"

    Do this only for applications in the subsegment <subsegment>{subsegment}</subsegment>

    Every X in the spreadsheet should correspond to a statement.
    Include a response for every single X in this subsegment. DO NOT leave any out or abbreviate, even if the response is very long.
    Do not include any preamble or explanation, just give the list of statements.

    <spreadsheet>{raw_output}</spreadsheet>
  """)
  print(str(response))
  all_responses += str(response) + "\n"

Aparentia Arquitectónico es apropiado para losas en estacionamientos de edificación vertical en comercio.
Duramax es apropiado para losas en estacionamientos de edificación vertical en comercio.
Promptis Resistencia Acelerada es apropiado para losas en estacionamientos de edificación vertical en comercio.
Hidratium es apropiado para losas en estacionamientos de edificación vertical en comercio.
Ingenia Trabajabilidad Extendida es apropiado para losas en estacionamientos de edificación vertical en comercio.
Pisocret Baja Contracción es apropiado para losas en estacionamientos de edificación vertical en comercio.
Pavicrete MR es apropiado para losas en estacionamientos de edificación vertical en comercio.
Isularis Logero es apropiado para losas en estacionamientos de edificación vertical en comercio.
Reducrack ST es apropiado para losas en estacionamientos de edificación vertical en comercio.
Reducrack Steal Deck es apropiado para losas en estacionamientos de edificación vertical en come

In [ ]:
print(all_responses)

Aparentia Arquitectónico es apropiado para losas en estacionamientos de edificación vertical en comercio.
Duramax es apropiado para losas en estacionamientos de edificación vertical en comercio.
Promptis Resistencia Acelerada es apropiado para losas en estacionamientos de edificación vertical en comercio.
Hidratium es apropiado para losas en estacionamientos de edificación vertical en comercio.
Ingenia Trabajabilidad Extendida es apropiado para losas en estacionamientos de edificación vertical en comercio.
Pisocret Baja Contracción es apropiado para losas en estacionamientos de edificación vertical en comercio.
Pavicrete MR es apropiado para losas en estacionamientos de edificación vertical en comercio.
Isularis Logero es apropiado para losas en estacionamientos de edificación vertical en comercio.
Reducrack ST es apropiado para losas en estacionamientos de edificación vertical en comercio.
Reducrack Steal Deck es apropiado para losas en estacionamientos de edificación vertical en come

We get a set of thousands of statements, one for each X in the spreadsheet.

In [ ]:
print(len(str(all_responses).split("\n")))

2954


We now convert our combined statements into embeddings. This means we only need to do our conversion step one time, and thereafter the statements are stored in Mongo.

In [ ]:
from llama_index.core import Document

document = Document(text=all_responses)

index.insert(document)

We now create a query engine from the index. Since we only want concise answers, we can switch back to our LLM with much lower `max_tokens` value.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm # this is the short one
query_engine = index.as_query_engine()

We provide our sample query in Spanish and get back a Spanish-language answer:

In [ ]:
response = query_engine.query("para la especialidad de comercio para la especial de hotelería qué concreto me recomiendas para muros de un edificio?")
print(response)

Para muros de un edificio en hotelería comercial, puedo recomendarte varias opciones de concreto que serían adecuadas:

- Aparentia Arquitectónico: ideal si buscas un acabado estético
- Duramax: ofrece gran durabilidad
- Promptis Resistencia Acelerada: cuando necesitas desarrollo rápido de resistencia
- Promptis Sin mano de obra: facilita la instalación
- Evolution Autocompactable: excelente para estructuras complejas
- Fortis Alta Resistencia: cuando requieres mayor capacidad estructural
- Hidratium: buena opción general
- Ingenia Revenimiento Total: mejora la trabajabilidad
- Ingenia Trabajabilidad Extendida: mantiene propiedades por más tiempo
- Ingenia Mortero Estabilizado: para aplicaciones específicas
- Reducrack ST: ayuda a reducir agrietamientos

Estos productos están específicamente diseñados para cumplir con los requerimientos de muros en edificaciones hoteleras comerciales, ofreciendo diferentes características según tus necesidades particulares de construcción.
